In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
import dotenv
sys.path.append("../../")

dotenv.load_dotenv("../.env")

True

In [3]:
USERNAME = "mytestuser1"
PASSWORD = "mytestpassword"
EMAIL = "mytestuser1@gmail.com"
FULL_NAME = "My Test User"

In [4]:
# first of all we sign up to the atria hub to manage our datasets and models
from atria_hub.hub import AtriaHub
hub = AtriaHub()
try:
    user = hub.auth.sign_up(email=EMAIL, password=PASSWORD, full_name=FULL_NAME, username=USERNAME)
except Exception:
    pass 

In [5]:
# after a successful sign up, we can initialize the hub with our login credentials
user = hub.auth.sign_in(email=EMAIL, password=PASSWORD)
hub.initialize()


Sign-in successful for mytestuser1@gmail.com


In [6]:
# now we first load the cifar10 dataset from the atria datasets predefiend registry
from atria_datasets import AtriaImageDataset
dataset = AtriaImageDataset.load_from_registry("cifar10")
# now lets upload the dataset to the atria hub
dataset.upload_to_hub('test-cifar10-5', branch='test')


/mnt/hephaistos/projects/atria_agent/atria/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-19 04:19:27,435	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[2025-07-19 04:19:27][atria_datasets.core.dataset.atria_dataset][INFO] Loading dataset cifar10 from registry.
[2025-07-19 04:19:27][atria_datasets.core.dataset.atria_dataset][INFO] Loading cached split train from /mnt/hephaistos/.atria/datasets/cifar10/main-01eefb81/delta/train
[2025-07-19 04:19:27][atria_datasets.core.dataset.atria_dataset][INFO] Loading cached split test from /mnt/hephaistos/.atria/datasets/cifar10/main-01eefb81/delta/test
[2025-07-19 04:19:28][atria_datasets.core.dataset.atria_dataset][WARNING] Split validation not found.

RuntimeError: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.

In [ ]:
# now lets upload the dataset to the atria hub
dataset.upload_to_hub('test-cifar10-4', branch='test')

[2025-07-19 04:15:23][atria_datasets.core.dataset.atria_dataset][INFO] Uploading dataset Cifar10 to hub with name test-cifar10-4 and config test.
[2025-07-19 04:15:23][atria_datasets.core.dataset.atria_dataset][ERROR] Failed to upload dataset to hub: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.


dir_ls mytestuser1-26c51e41f3db44e1/test/ [{'checksum': 'd41d8cd98f00b204e9800998ecf8427e', 'content-type': 'text/markdown', 'mtime': 1752871159, 'name': 'mytestuser1-26c51e41f3db44e1/test/card.md', 'size': 0, 'type': 'file'}, {'name': 'mytestuser1-26c51e41f3db44e1/test/conf/', 'size': 0, 'type': 'directory'}, {'name': 'mytestuser1-26c51e41f3db44e1/test/delta/', 'size': 0, 'type': 'directory'}, {'checksum': '4b44958f6b5d0ec43b61cefee30fb7d8', 'content-type': 'application/octet-stream', 'mtime': 1752891124, 'name': 'mytestuser1-26c51e41f3db44e1/test/metadata.yaml', 'size': 235, 'type': 'file'}]


RuntimeError: This dataset already contains uploaded files in branch 'test'. Please use a different branch name or delete the existing files.

In [12]:
# now lets upload the dataset to the atria hub
from atria_datasets.core.dataset.atria_dataset import AtriaImageDataset
dataset = AtriaImageDataset.load_from_hub(
    name=f"{hub.auth.username}/test-cifar10-4/test", streaming=True
)
print(dataset.metadata)
x = dataset.train[0]
print(x)

[2025-07-19 04:23:03][atria_datasets.core.dataset.atria_hub_dataset][INFO] Streaming dataset split train from lakefs://mytestuser1-26c51e41f3db44e1/test/delta/train/
[2025-07-19 04:23:03][atria_datasets.core.dataset.atria_hub_dataset][INFO] Streaming dataset split test from lakefs://mytestuser1-26c51e41f3db44e1/test/delta/test/
[2025-07-19 04:23:03][atria_datasets.core.dataset.atria_dataset][WARNING] Split validation not found. Skipping.


homepage=None description='CIFAR-10 dataset' license=None citation=None dataset_labels=DatasetLabels(
    classification=[
        'airplane',
        'automobile',
        'bird',
        'cat',
        'deer',
        'dog',
        'frog',
        'horse',
        'ship',
        'truck'
    ],
    ser=None,
    layout=None
)
ImageInstance(
    index=0,
    sample_id='09447729-0f39-4fb3-855f-9aedf183d106',
    image=Image(
        content=<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7C9BAC191790>,
        source_width=32,
        source_height=32,
        width=32,
        height=32,
        channels=3
    ),
    gt=GroundTruth(
        classification=ClassificationGT(label=Label(value=6, name='frog'))
    )
)


In [13]:
# now we build a resnet50 model pipeline for image classification from the atria models registry
from atria_models.pipelines.atria_model_pipeline import AtriaModelPipeline
model_pipeline = AtriaModelPipeline.load_from_registry(
    pipeline_name="image_classification",
    model_name="resnet50",
    dataset_metadata=dataset.metadata,
)



[2025-07-19 04:23:15][atria_models.core.timm_model][INFO] Setting up model TimmModel(model_name='resnet50', num_classes=10)


In [16]:
# now we upload the model pipeline to the atria hub
model_pipeline.upload_to_hub(
    name="resnet50",
    branch="main",
)

[2025-07-19 04:24:18][atria_models.pipelines.atria_model_pipeline][INFO] Uploading model ImageClassificationPipeline to hub with name resnet50 and config main.


In [21]:
# now we load the model pipeline from the hub
name = f"{hub.auth.username}/resnet50/main"
print(name)
model_pipeline = AtriaModelPipeline.load_from_hub(
    name=f"{hub.auth.username}/resnet50/main",
)

[2025-07-19 04:28:46][atria_models.pipelines.atria_model_pipeline][INFO] Loading model mytestuser1/resnet50/main from hub.


mytestuser1/resnet50/main


[2025-07-19 04:28:47][atria_models.core.timm_model][INFO] Setting up model TimmModel(model_name='resnet50', num_classes=10)


In [20]:
from atria_core.types.factory import ImageInstanceFactory
from atria_core.types import GroundTruth
model_pipeline.enable_runtime_transforms()
image = ImageInstanceFactory.build()
image.gt = GroundTruth(classification = image.gt.classification)
model_pipeline.training_step([image])

[3, 5, 8, 4, 9, 10, 7, 10, 3, 0]
[6, 7, 7, 4, 3, 6, 7, 0, 3, 4]


ClassificationModelOutput(loss=tensor(2.6781, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0011, -0.3447,  0.1753,  0.0382, -0.1193,  0.2478, -0.0659,  0.1210,
         -0.2859,  0.3294]], grad_fn=<AddmmBackward0>), prediction=None, label=Label(value=tensor([1]), name=['participant']))

In [ ]:
# from omegaconf import OmegaConf
# print(json.dumps(OmegaConf.to_container(model_pipeline.config), indent=2, sort_keys=True))

In [ ]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline

# print(model_pipeline.config)
# model_pipeline.upload_to_hub(
#     "mymodel"
# )

# loaded = ImageClassificationPipeline.load_from_hub(
#     "mymodel", 
# )

In [ ]:
# loaded.state_dict()['dataset_metadata'] = model_pipeline.state_dict()['dataset_metadata']
# loaded.state_dict()['config'] = model_pipeline.state_dict()['config']
# print(loaded.state_dict()['config'])

In [ ]:
# from atria_models.pipelines.classification.image import ImageClassificationPipeline
# from omegaconf import OmegaConf
# from atria_models.registry import MODEL
# from atria_models.core.timm_model import TimmModel
# from rich.pretty import pretty_repr
# import json
# model = TimmModel(model_name="resnet50")
# pipeline = ImageClassificationPipeline(model=model, checkpoint_configs='test')
# print(json.dumps(pipeline._config, indent=2))